# Outages Exploratory Analysis

**Name(s)**: Claire Wang, Emily Yip

**Website Link**: (your website link)

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

#from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [70]:
outages = pd.read_csv('data/outages.csv')
outages.columns = ['index'] + list(outages.iloc[4].iloc[1:])
outages = outages.iloc[6:]
outages = outages.reset_index().drop(columns=['level_0', 'index'])
outages

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 01, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983
1,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983
2,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983
3,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983
4,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,"Tuesday, December 06, 2011",...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.59964921,2.401765255,2.401765255
1530,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.59964921,2.401765255,2.401765255
1531,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,"Saturday, August 29, 2009",...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.30774418,1.692255822,1.692255822
1532,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,"Saturday, August 29, 2009",...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.30774418,1.692255822,1.692255822


In [55]:
outages['OUTAGE.START.DATE'].iloc[0].split(', ')[1:]

['July 01', '2011']

## Step 2: Data Cleaning and Exploratory Data Analysis

In [77]:
outages['OUTAGE.START.TIME'].isna().sum()

np.int64(9)

In [81]:
# clean start and restoration dates. Drop missing values because there aren't many
# turn into pandas date time
def clean_time(df):
    df = df.dropna(subset = ['OUTAGE.START.DATE'])
    df['OUTAGE.START.DATE'] = df['OUTAGE.START.DATE'].transform(lambda x: x.split(', ')[1:])
    date_series = df['OUTAGE.START.DATE'].transform(lambda x: x[0] + ', ' + x[1])
    df['exact_start_time'] = date_series + ' ' + df['OUTAGE.START.TIME']
    df['exact_start_time'] = pd.to_datetime(df['exact_start_time'], format="%B %d, %Y %I:%M:%S %p")

    df = df.dropna(subset = ['OUTAGE.RESTORATION.DATE'])
    df['OUTAGE.RESTORATION.DATE'] = df['OUTAGE.RESTORATION.DATE'].transform(lambda x: x.split(', ')[1:])
    date_series = df['OUTAGE.RESTORATION.DATE'].transform(lambda x: x[0] + ', ' + x[1])
    df['exact_restoration_time'] = date_series + ' '+ df['OUTAGE.RESTORATION.TIME']
    df['exact_restoration_time'] = pd.to_datetime(df['exact_restoration_time'], format="%B %d, %Y %I:%M:%S %p")
    
    df = df.drop(columns=['OUTAGE.RESTORATION.DATE', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME',
                          'OUTAGE.RESTORATION.TIME'])
    return df

outages_cleaned = (outages
                   .pipe(clean_time))
outages_cleaned

/var/folders/r6/j2xslqfj0lg0bxy1nj8b6fzm0000gn/T/ipykernel_6801/1687438539.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OUTAGE.START.DATE'] = df['OUTAGE.START.DATE'].transform(lambda x: x.split(', ')[1:])
/var/folders/r6/j2xslqfj0lg0bxy1nj8b6fzm0000gn/T/ipykernel_6801/1687438539.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['exact_start_time'] = date_series + ' ' + df['OUTAGE.START.TIME']
/var/folders/r6/j2xslqfj0lg0bxy1nj8b6fzm0000gn/T/ipykernel_6801/1687438539.py:8: SettingWithCopyWarn

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,exact_start_time,exact_restoration_time
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2011-07-01 17:00:00,2011-07-03 20:00:00
1,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,...,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2014-05-11 18:38:00,2014-05-11 18:39:00
2,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2010-10-26 20:00:00,2010-10-28 22:00:00
3,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2012-06-19 04:30:00,2012-06-20 23:00:00
4,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,1527,2016,3,Idaho,ID,WECC,Northwest,1.6,warm,intentional attack,...,2216.8,2004.7,5.6,0.6,0.19,98.89193361,1.108066388,1.108066388,2016-03-08 00:00:00,2016-03-08 00:00:00
1528,1529,2016,7,Idaho,ID,WECC,Northwest,-0.3,normal,system operability disruption,...,2216.8,2004.7,5.6,0.6,0.19,98.89193361,1.108066388,1.108066388,2016-07-19 15:45:00,2016-07-19 19:25:00
1529,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,...,2192.2,1868.2,3.9,0.27,0.1,97.59964921,2.401765255,2.401765255,2011-12-06 08:00:00,2011-12-06 20:00:00
1531,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,...,2038.3,1905.4,4.7,0.3,0.15,98.30774418,1.692255822,1.692255822,2009-08-29 22:54:00,2009-08-29 23:53:00


## Step 3: Assessment of Missingness

In [ ]:
# TODO

## Step 4: Hypothesis Testing

In [ ]:
# TODO

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO